In [ ]:
!pip install yfinance

# Collect the data

In [ ]:
import yfinance as yf

# Define the stock tickers
# for PE fund
tickers = ["APO", "BX", "BN", "KKR", "CG", "ARES", "OWL", "PAX", "SPY", "BRK-B"] 

# for AI fund
#tickers = ["MSFT", "GOOG", "AMZN", "SNOW", "PLTR", "SPY", "BRK-B"]

MA_WINDOW = 30 #number of days for the moving average window
START_DATE = "2019-08-31"
END_DATE = "2024-08-31"

# Download historical data for the last 5 years, including dividends
data = yf.download(tickers, start=START_DATE, end=END_DATE, actions=True)['Adj Close']

# Calculate the N-day moving average
data_ma = data.rolling(window=MA_WINDOW).mean()

# Normalize the data to start at 100 for easy comparison
normalized_data_ma = data_ma / data_ma.iloc[0] * 100  # start normalization after the 7th day

# Create the synthetic fund with the "PEF" ticker as an equally weighted average of the other tickers (excluding SPY)
# Normalize the data to start at 100 for easy comparison
normalized_data = data / data.iloc[0] * 100

# Create the "PEF" ticker as an equally weighted portfolio at the start
initial_weights = 1 / len(tickers[:-2])  # Exclude SPY and BRKB from the calculation
pef_series = (normalized_data[tickers[:-2]] * initial_weights).sum(axis=1)

# Calculate the N-day moving average for all stocks including "PEF"
normalized_data_ma["SYN-FUND1"] = normalized_data_ma[tickers[:-2]].mean(axis=1) #
normalized_data_ma["SYN-FUND2"] = pef_series.rolling(window=MA_WINDOW).mean() #

# Add BRKB to the tickers for plotting
tickers_with_pef = tickers + ["SYN-FUND1", "SYN-FUND2"]

In [ ]:
data_ma

# Plot the data

In [ ]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add traces for each stock
for ticker in tickers:
    fig.add_trace(go.Scatter(x=normalized_data_ma.index, 
                             y=normalized_data_ma[ticker],
                             mode='lines', 
                             name=ticker))

# Add the PEF trace with a dotted black line
fig.add_trace(go.Scatter(x=normalized_data_ma.index, 
                         y=normalized_data_ma["SYN-FUND1"],
                         mode='lines', 
                         name="SYN-FUND1", 
                         line=dict(color='black', width=4, dash='dot')))

fig.add_trace(go.Scatter(x=normalized_data_ma.index, 
                         y=normalized_data_ma["SYN-FUND2"],
                         mode='lines', 
                         name="SYN-FUND2", 
                         line=dict(color='grey', width=4, dash='dot')))

# Customize the layout
fig.update_layout(
    title="Performance of Selected Stocks Including Dividends ("+str(MA_WINDOW)+"-Day Moving Average)",
    xaxis_title="Date",
    yaxis_title="Normalized Price (Starting at 100)",
    legend_title="Stocks",
    hovermode="x unified",
    width=1400,  # Set width to 14 inches (14 * 100 pixels per inch)
    height=800,  # Set height to 8 inches (8 * 100 pixels per inch)
    plot_bgcolor='white',  # Set the plot background to white
    paper_bgcolor='white'  # Set the outer background to white
)

# Show the interactive plot
fig.show()

# P/E and forward P/E and other metrics

In [ ]:
import yfinance as yf
import pandas as pd

with open("tickers.txt", "r") as file:
    tickers = [line.strip() for line in file]

# Initialize an empty list to store data
data = []

# Collect the P/E and forward P/E for each company
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        company_name = stock.info.get("longName", "N/A")
        pe_ratio = stock.info.get("trailingPE", "N/A")
        forward_pe = stock.info.get("forwardPE", "N/A")
        revenue_growth = stock.info.get("revenueGrowth", "N/A")
        beta = stock.info.get("beta", "N/A")
        data.append({"Ticker": ticker, 
                     "Company Name": company_name, 
                     "P/E Ratio": pe_ratio, 
                     "Forward P/E": forward_pe, 
                     "Revenue Growth": revenue_growth, 
                     "Beta": beta})
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Load data into a DataFrame
df = pd.DataFrame(data)

In [ ]:
display(df)